<a href="https://colab.research.google.com/github/syma-afsha/AI_LAB/blob/main/MirMohibullahSazid_DND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import numpy as np


In [62]:
#Necessary imports
import torch
import torch.nn as nn
import numpy as np

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [64]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

In [65]:
# Cell 0.3 -> INPUT (DO NOT EDIT THIS CELL!)
input_tokens =  torch.randint(0, 5000, (4, 20))  # [B=4, T=20]
print(input_tokens.shape)

torch.Size([4, 20])


In [66]:
embedding=nn.Embedding(num_embeddings=5000, embedding_dim=128)
embedded = embedding(input_tokens)
print(f"After embedding: {embedded.shape}")
dropout = nn.Dropout(0.1)
input_tokens= dropout(embedded)

After embedding: torch.Size([4, 20, 128])


In [67]:
class MultiKernelCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3, padding=1)
        self.relu=nn.ReLU()
        self.pool1=nn.AdaptiveMaxPool1d(output_size=1) #outputzise
        self.conv2=nn.Conv1d(in_channels=128, out_channels=32, kernel_size=5, padding=2)
        self.batchnorm=nn.BatchNorm1d(num_features=32)
        self.pool2=nn.MaxPool1d(kernel_size=15, stride=15)
        self.conv3=nn.Conv1d(in_channels=128, out_channels=32, kernel_size=7, padding=2) #padding change
        self.pool3=nn.MaxPool1d(kernel_size=10, stride=10)
        self.dropout2=nn.Dropout(0.1)


    def forward(self, x):
        # ADD YOUR CODE HERE
        #transpose
        x=x.permute(0,2,1)
        print(f"After transpose: {x.shape}")
        x1=self.conv1(x)
        x1=self.relu(x1)
        print(f"After conv1: {x1.shape}")
        x1=self.pool1(x1)
        print(f"After pool1: {x1.shape}")
        #squeeze(-1)
        x1=x1.squeeze(-1)
        print(f"After squeeze: {x1.shape}")

        x2=self.conv2(x)
        x2=self.batchnorm(x2)
        print(f"After conv2: {x2.shape}")
        x2=self.pool2(x2)
        print(f"After pool2: {x2.shape}")
        x2=x2.squeeze(-1)
        print(f"After squeeze: {x2.shape}")
        x3=self.conv3(x)
        x3=self.relu(x3)
        print(f"After conv3: {x3.shape}")
        x3=self.pool3(x3)
        print(f"After pool3: {x3.shape}")
        x3=x3.squeeze(-1)
        print(f"After squeeze: {x3.shape}")
        x=torch.cat([x1,x2,x3],dim=1)
        print(f"After concat: {x.shape}")
        x=self.dropout2(x)
        print(f"After dropout: {x.shape}")



        return x
multi_kernel=MultiKernelCNN()
multi_kernel_cnn=multi_kernel(input_tokens)
print(f"Final MultikernelCnn,{multi_kernel_cnn.shape}")


After transpose: torch.Size([4, 128, 20])
After conv1: torch.Size([4, 32, 20])
After pool1: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv2: torch.Size([4, 32, 20])
After pool2: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv3: torch.Size([4, 32, 18])
After pool3: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After concat: torch.Size([4, 96])
After dropout: torch.Size([4, 96])
Final MultikernelCnn,torch.Size([4, 96])


In [68]:
class BiLSTMEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # ADD YOUR CODE HERE
        self.bilstm=nn.LSTM(input_size=128, hidden_size=256, batch_first=True, bidirectional=True)


    def forward(self, x):
        # ADD YOUR CODE HERE
        _, (hidden,_)=self.bilstm(x)
        print(f"After bilstm: {x.shape}")
        hidden_forward=hidden[-2, :, :]
        hidden_backward=hidden[-1, :, :]
        hidden=torch.cat([hidden_forward, hidden_backward], dim=1)
        print(f"After concat: {hidden.shape}")


        return hidden
bilstm_encoder=BiLSTMEncoder()
bilstm_encoder_output=bilstm_encoder(input_tokens)
print(f"Final BiLSTMEncoder,{bilstm_encoder_output.shape}")

After bilstm: torch.Size([4, 20, 128])
After concat: torch.Size([4, 512])
Final BiLSTMEncoder,torch.Size([4, 512])


In [69]:
class TextClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # ADD YOUR CODE HERE
        self.multi_kernel=MultiKernelCNN()
        self.bilstm_encoder=BiLSTMEncoder()
        self.dropout=nn.Dropout(0.1)

        self.linear1=nn.Linear(in_features=608, out_features=256)
        self.relu=nn.ReLU()

        self.linear2=nn.Linear(in_features=256, out_features=4)


    def forward(self, x):
        # ADD YOUR CODE HERE
        x1=self.multi_kernel(x)
        x2=self.bilstm_encoder(x)
        x=torch.cat([x1,x2], dim=1)
        print(f"Before Sequential concat: {x.shape}")
        x=self.linear1(x)
        print(f"Sequential output: {x.shape}")
        x=self.relu(x)
        x=self.dropout(x)
        x=self.linear2(x)
        print(f"Final Output: {x.shape}")

        return x
text_classifier=TextClassifier()
text_classifier_output=text_classifier(input_tokens)

After transpose: torch.Size([4, 128, 20])
After conv1: torch.Size([4, 32, 20])
After pool1: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv2: torch.Size([4, 32, 20])
After pool2: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv3: torch.Size([4, 32, 18])
After pool3: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After concat: torch.Size([4, 96])
After dropout: torch.Size([4, 96])
After bilstm: torch.Size([4, 20, 128])
After concat: torch.Size([4, 512])
Before Sequential concat: torch.Size([4, 608])
Sequential output: torch.Size([4, 256])
Final Output: torch.Size([4, 4])


In [70]:
#DO NOT MODIFY THIS CELL

print("ComplexTextClassifier:")
complex_text_classifier = TextClassifier()
text_embedding = complex_text_classifier(input_tokens)

try:
    assert text_embedding.shape == (4, 4), "Classification output shape is incorrect."
    print("\n🎉 Congratulations! Your implementation is correct. You passed the minimum requirement! 🎉")
except AssertionError as e:
    print(f"\n❌ Error: {e}")

ComplexTextClassifier:
After transpose: torch.Size([4, 128, 20])
After conv1: torch.Size([4, 32, 20])
After pool1: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv2: torch.Size([4, 32, 20])
After pool2: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After conv3: torch.Size([4, 32, 18])
After pool3: torch.Size([4, 32, 1])
After squeeze: torch.Size([4, 32])
After concat: torch.Size([4, 96])
After dropout: torch.Size([4, 96])
After bilstm: torch.Size([4, 20, 128])
After concat: torch.Size([4, 512])
Before Sequential concat: torch.Size([4, 608])
Sequential output: torch.Size([4, 256])
Final Output: torch.Size([4, 4])

🎉 Congratulations! Your implementation is correct. You passed the minimum requirement! 🎉
